In [10]:
import pandas as pd
import glob
import os

LEAD_TITLE = 'India'
LEAD = '1585'

folder_path = f'c:\\Workspace\\pre-planning\\ATT3\\{LEAD}\\'

all_files = glob.glob(os.path.join(folder_path, "*.csv"))

df_list = []
for file in all_files:
    df = pd.read_csv(file)  
    df_list.append(df)

concatenated_df = pd.concat(df_list, ignore_index=True)

concatenated_df.to_csv(f'{LEAD}concatenated_file.csv', index=False)

com = pd.read_csv(f'{LEAD}concatenated_file.csv')
print(com['status'].value_counts())



# Rename status column values
status_mapping = {
    'realized': 'Realized',
    'matched': 'Accepted by Host',
    'an': 'Accepted',
    'approved': 'Approved',
    'finished': 'Finished',
    'completed': 'Completed',
    'applied': 'Applied',
    'remote': 'Remote Realized',
    'o': 'Open',
    'i': 'Open',
    'total': 'Total',
    'accepted': 'Accepted L'
}

com['status'] = com['status'].map(status_mapping)
com['Month in Dates'] = pd.to_datetime(com['Month'])
com['status2'] = com['status']

print(com)


# Create a PeriodIndex with monthly frequency
period_index = pd.PeriodIndex(com['Month in Dates'], freq='M')

# Convert 'Month in Dates' to datetime
com['Month in Dates'] = pd.to_datetime(com['Month in Dates'])

# Define a mapping dictionary for quarters
month_mapping = {
    1: 'JAN',
    2: 'FEB',
    3: 'MAR',
    4: 'APR',   
    5: 'MAY',
    6: 'JUN',
    7: 'JUL',
    8: 'AUG',
    9: 'SEP',
    10: 'OCT',
    11: 'NOV',
    12: 'DEC',
}

# Extract month numbers and map them to month names
com['Months for Analysis'] = com['Month in Dates'].dt.month.map(month_mapping)

# Function to determine term based on date
def determine_term(date_str):
    date = pd.to_datetime(date_str, format='%Y-%m-%d')
    year = date.year
    if date.month >= 2:
        term = f"{year % 100}.{(year + 1) % 100}"
    else:
        term = f"{(year - 1) % 100}.{year % 100}"
    return term

# Adding 'Term' column to DataFrame
com['Term'] = com['Month in Dates'].apply(determine_term)

com.to_csv(f'{LEAD}final_processed.csv', index=False)

# function_is = 'oGTa'
# term = '20.21'
def get_summary(df, function_new, term):
    
    df['Month in Dates'] = pd.to_datetime(df['Month in Dates'])
    col_to_show = ['status','status2', 'APP','Month in Dates', 'function_new', 'Term', 'Months for Analysis']
    # Filter the dataframe based on function and term
    filt = df[(df['function_new'] == function_new) & (df['Term'] == term)][col_to_show]
    pivot_df = filt.pivot_table(index='status', columns='Months for Analysis', values='APP', aggfunc='sum')
    pivot_df['Total'] = pivot_df.sum(axis=1)
    # a colum which has values 'ASL' for the whole column
    pivot_df['Entity'] = LEAD_TITLE
    pivot_df['Function'] = function_new
    pivot_df['Term'] = term
    pivot_df['Year'] = pivot_df['Term'].apply(lambda x: "20" + x.split('.')[0])
    pivot_df['EY Year'] = pivot_df['Entity'] + ' - ' + pivot_df['Year']

    

    rows_to_drop = ['Accepted L', 'Remote Realized', 'Accepted by Host']
    pivot_df = pivot_df.drop(rows_to_drop, errors='ignore')

    # Define the desired column order
    column_order = ['Entity', 'status2','Function','Term', 'Year','EY Year','FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC', 'JAN', 'Total']
    pivot_df = pivot_df.reindex(columns=column_order)

    # Define the desired row order
    row_order = ['Open','Applied', 'Accepted', 'Approved', 'Realized', 'Finished', 'Completed']
    pivot_df = pivot_df.reindex(row_order)
    
    return pivot_df

# get_summary(com, function_is, term)

import csv

functions = ['oGTa', 'iGTa', 'iGV', 'oGV', 'Total', 'oGTe', 'iGTe']
# terms = ['15.16', '16.17', '17.18', '18.19', '19.20','22.23', '23.24']
terms = ['15.16', '16.17', '17.18', '18.19', '19.20', '21.22', '22.23','23.24', '24.25']

# Open the CSV file in write mode
with open(f'{LEAD}Planning_Pivot.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    
    # List to store all summary DataFrames
    summary_dfs = []

    for func in functions:
        for term in terms:
            # Print and write the Function and Term identifiers
            print(f"Function: {func}, Term: {term}")
            
            # Get the summary DataFrame
            summary_df = get_summary(com, func, term)
            display(summary_df)
            summary_dfs.append(summary_df)
        
            
            # Write the Function and Term identifiers to the CSV
            csvwriter.writerow([f"Function: {func}, Term: {term}"])
            
            # Write the headers including the index name
            headers = ['Index'] + list(summary_df.columns)
            csvwriter.writerow(headers)
            
            # Write the DataFrame rows including the index
            for index, row in summary_df.iterrows():
                csvwriter.writerow([index] + list(row))
            
            # Add a blank line for separation (optional)
            csvwriter.writerow([])

        final_summary_df = pd.concat(summary_dfs, ignore_index=True)
        final_summary_df.reset_index(inplace=True)
        final_summary_df.to_csv(f'{LEAD_TITLE}_{LEAD}summary_data_for_peak.csv', index=False)

print(f'{LEAD_TITLE}_{LEAD}Data written to summary_data.csv successfully.')



realized     572
an           528
approved     528
finished     528
completed    528
applied      528
remote       528
o            264
i            264
total        264
accepted      44
Name: status, dtype: int64
        status function function_new          Month  APP Month in Dates  \
0     Realized  Old iGT         iGTa   January 2021  0.0     2021-01-01   
1     Realized  Old iGV          iGV   January 2021  0.0     2021-01-01   
2     Realized     iGTa         iGTa   January 2021  2.0     2021-01-01   
3     Realized      iGV          iGV   January 2021  0.0     2021-01-01   
4     Realized     iGTe         iGTe   January 2021  1.0     2021-01-01   
...        ...      ...          ...            ...  ...            ...   
4571     Total    Total        Total  November 2024  0.0     2024-11-01   
4572     Total    Total        Total  November 2024  0.0     2024-11-01   
4573  Realized    Total        Total  November 2024  0.0     2024-11-01   
4574     Total    Total        Total

Months for Analysis,Entity,status2,Function,Term,Year,EY Year,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,JAN,Total
status,,,,,,,,,,,,,,,,,,,
Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Applied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accepted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Approved,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Realized,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Finished,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Function: oGTa, Term: 16.17


Months for Analysis,Entity,status2,Function,Term,Year,EY Year,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,JAN,Total
status,,,,,,,,,,,,,,,,,,,
Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Applied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accepted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Approved,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Realized,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Finished,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Function: oGTa, Term: 17.18


...

Function: iGTe, Term: 23.24


Months for Analysis,Entity,status2,Function,Term,Year,EY Year,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,JAN,Total
status,,,,,,,,,,,,,,,,,,,
Open,India,NaN,iGTe,23.24,2023,India - 2023,6.0,16.0,27.0,22.0,9.0,19.0,20.0,13.0,17.0,20.0,NaN,25.0,194.0
Applied,India,NaN,iGTe,23.24,2023,India - 2023,74.0,113.0,152.0,123.0,90.0,75.0,139.0,104.0,87.0,131.0,NaN,142.0,1230.0
Accepted,India,NaN,iGTe,23.24,2023,India - 2023,4.0,10.0,13.0,11.0,8.0,4.0,6.0,8.0,3.0,5.0,NaN,5.0,77.0
Approved,India,NaN,iGTe,23.24,2023,India - 2023,5.0,7.0,4.0,5.0,8.0,1.0,4.0,5.0,2.0,2.0,NaN,2.0,45.0
Realized,India,NaN,iGTe,23.24,2023,India - 2023,0.0,0.0,3.0,4.0,4.0,7.0,2.0,5.0,0.0,0.0,NaN,5.0,30.0
Finished,India,NaN,iGTe,23.24,2023,India - 2023,0.0,4.0,2.0,1.0,4.0,5.0,4.0,4.0,2.0,8.0,NaN,2.0,36.0
Completed,India,NaN,iGTe,23.24,2023,India - 2023,0.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,0.0,3.0,NaN,2.0,11.0


Function: iGTe, Term: 24.25


Months for Analysis,Entity,status2,Function,Term,Year,EY Year,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,JAN,Total
status,,,,,,,,,,,,,,,,,,,
Open,India,NaN,iGTe,24.25,2024,India - 2024,7.0,32.0,21.0,18.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,79.0
Applied,India,NaN,iGTe,24.25,2024,India - 2024,148.0,288.0,217.0,224.0,23.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,900.0
Accepted,India,NaN,iGTe,24.25,2024,India - 2024,6.0,17.0,15.0,22.0,7.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,67.0
Approved,India,NaN,iGTe,24.25,2024,India - 2024,8.0,11.0,15.0,18.0,3.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,55.0
Realized,India,NaN,iGTe,24.25,2024,India - 2024,3.0,0.0,2.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,11.0
Finished,India,NaN,iGTe,24.25,2024,India - 2024,1.0,1.0,0.0,3.0,5.0,6.0,0.0,3.0,0.0,2.0,NaN,NaN,21.0
Completed,India,NaN,iGTe,24.25,2024,India - 2024,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0


India_1585Data written to summary_data.csv successfully.


In [1]:
import pandas as pd
import glob
import os

In [2]:
LEAD = '1585'

In [3]:
folder_path = f'c:\\Workspace\\pre-planning\\ATT3\\{LEAD}\\'

all_files = glob.glob(os.path.join(folder_path, "*.csv"))

df_list = []
for file in all_files:
    df = pd.read_csv(file)  
    df_list.append(df)

concatenated_df = pd.concat(df_list, ignore_index=True)

concatenated_df.to_csv(f'{LEAD}concatenated_file.csv', index=False)

In [4]:
com = pd.read_csv(f'{LEAD}concatenated_file.csv')
print(com['status'].value_counts())
com

realized     1092
matched      1008
an           1008
approved     1008
finished     1008
completed    1008
applied      1008
remote       1008
o             504
i             504
total         504
accepted       84
Name: status, dtype: int64


,status,function,function_new,Month,APP
0,matched,oGE,oGTa,January 2016,0.0
1,realized,Old iGT,iGTa,January 2016,3.0
2,realized,Old iGV,iGV,January 2016,192.0
3,matched,oGTe,oGTe,January 2016,0.0
4,matched,oGTa,oGTa,January 2016,0.0
...,...,...,...,...,...
9739,total,Total,Total,December 2024,0.0
9740,total,Total,Total,December 2024,0.0
9741,realized,Total,Total,December 2024,0.0
9742,total,Total,Total,December 2024,0.0


In [5]:

# Rename status column values
status_mapping = {
    'realized': 'Realized',
    'matched': 'Accepted',
    'an': 'Accepted by Host',
    'approved': 'Approved',
    'finished': 'Finished',
    'completed': 'Completed',
    'applied': 'Applied',
    'remote': 'Remote Realized',
    'o': 'Open',
    'i': 'Open',
    'total': 'Total',
    'accepted': 'Accepted L'
}

com['status'] = com['status'].map(status_mapping)
com['Month in Dates'] = pd.to_datetime(com['Month'])
com

,status,function,function_new,Month,APP,Month in Dates
0,Accepted,oGE,oGTa,January 2016,0.0,2016-01-01
1,Realized,Old iGT,iGTa,January 2016,3.0,2016-01-01
2,Realized,Old iGV,iGV,January 2016,192.0,2016-01-01
3,Accepted,oGTe,oGTe,January 2016,0.0,2016-01-01
4,Accepted,oGTa,oGTa,January 2016,0.0,2016-01-01
...,...,...,...,...,...,...
9739,Total,Total,Total,December 2024,0.0,2024-12-01
9740,Total,Total,Total,December 2024,0.0,2024-12-01
9741,Realized,Total,Total,December 2024,0.0,2024-12-01
9742,Total,Total,Total,December 2024,0.0,2024-12-01


In [6]:
# Create a PeriodIndex with monthly frequency
period_index = pd.PeriodIndex(com['Month in Dates'], freq='M')

# Convert 'Month in Dates' to datetime
com['Month in Dates'] = pd.to_datetime(com['Month in Dates'])

# Define a mapping dictionary for quarters
month_mapping = {
    1: 'JAN',
    2: 'FEB',
    3: 'MAR',
    4: 'APR',   
    5: 'MAY',
    6: 'JUN',
    7: 'JUL',
    8: 'AUG',
    9: 'SEP',
    10: 'OCT',
    11: 'NOV',
    12: 'DEC',
}

# Extract month numbers and map them to month names
com['Months for Analysis'] = com['Month in Dates'].dt.month.map(month_mapping)

com

,status,function,function_new,Month,APP,Month in Dates,Months for Analysis
0,Accepted,oGE,oGTa,January 2016,0.0,2016-01-01,JAN
1,Realized,Old iGT,iGTa,January 2016,3.0,2016-01-01,JAN
2,Realized,Old iGV,iGV,January 2016,192.0,2016-01-01,JAN
3,Accepted,oGTe,oGTe,January 2016,0.0,2016-01-01,JAN
4,Accepted,oGTa,oGTa,January 2016,0.0,2016-01-01,JAN
...,...,...,...,...,...,...,...
9739,Total,Total,Total,December 2024,0.0,2024-12-01,DEC
9740,Total,Total,Total,December 2024,0.0,2024-12-01,DEC
9741,Realized,Total,Total,December 2024,0.0,2024-12-01,DEC
9742,Total,Total,Total,December 2024,0.0,2024-12-01,DEC


In [16]:
# Function to determine term based on date
def determine_term(date_str):
    date = pd.to_datetime(date_str, format='%Y-%m-%d')
    year = date.year
    if date.month >= 2:
        term = f"{year % 100}.{(year + 1) % 100}"
    else:
        term = f"{(year - 1) % 100}.{year % 100}"
    return term

# Adding 'Term' column to DataFrame
com['Term'] = com['Month in Dates'].apply(determine_term)


In [17]:
com

,status,function,function_new,Month,APP,Month in Dates,Months for Analysis,Term,Year
0,Accepted,oGE,oGTa,January 2016,0.0,2016-01-01,JAN,15.16,2015
1,Realized,Old iGT,iGTa,January 2016,3.0,2016-01-01,JAN,15.16,2015
2,Realized,Old iGV,iGV,January 2016,192.0,2016-01-01,JAN,15.16,2015
3,Accepted,oGTe,oGTe,January 2016,0.0,2016-01-01,JAN,15.16,2015
4,Accepted,oGTa,oGTa,January 2016,0.0,2016-01-01,JAN,15.16,2015
...,...,...,...,...,...,...,...,...,...
9739,Total,Total,Total,December 2024,0.0,2024-12-01,DEC,24.25,2024
9740,Total,Total,Total,December 2024,0.0,2024-12-01,DEC,24.25,2024
9741,Realized,Total,Total,December 2024,0.0,2024-12-01,DEC,24.25,2024
9742,Total,Total,Total,December 2024,0.0,2024-12-01,DEC,24.25,2024


In [19]:
com.to_csv(f'{LEAD}final_processed.csv', index=False)

In [27]:
# function_is = 'oGTa'
# term = '20.21'
def get_summary(df, function_new, term):
    
    df['Month in Dates'] = pd.to_datetime(df['Month in Dates'])
    col_to_show = ['status', 'APP','Month in Dates', 'function_new', 'Term', 'Months for Analysis','Year','EY Year']
    # Filter the dataframe based on function and term
    filt = df[(df['function_new'] == function_new) & (df['Term'] == term)][col_to_show]
    pivot_df = filt.pivot_table(index='status', columns='Months for Analysis', values='APP', aggfunc='sum')
    pivot_df['Total'] = pivot_df.sum(axis=1)
    # a colum which has values 'ASL' for the whole column
    pivot_df['Entity'] = LEAD
    #concat the year and EY year
    com['Year'] = com['Term'].apply(lambda x: "20" + x.split('.')[0])
    pivot_df['Year'] = pivot_df['Year'] + ' - ' + pivot_df['EY Year']


    rows_to_drop = ['Accepted L']
    pivot_df = pivot_df.drop(rows_to_drop, errors='ignore')

    # Define the desired column order
    column_order = ['Entity', 'Term', 'Year','EY Year', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC', 'JAN', 'Total']
    pivot_df = pivot_df.reindex(columns=column_order)

    # Define the desired row order
    row_order = ['Open','Applied', 'Accepted', 'Approved', 'Realized', 'Finished', 'Completed']
    pivot_df = pivot_df.reindex(row_order)
    
    return pivot_df

# get_summary(com, function_is, term)



In [28]:
com['Term'].unique()

array(['15.16', '16.17', '17.18', '18.19', '19.20', '21.22', '22.23',
       '23.24', '24.25'], dtype=object)

In [41]:
# functions = ['oGTa', 'iGTa', 'iGV', 'oGV', 'Total', 'oGTe', 'iGTe']
# # terms = ['15.16', '16.17', '17.18', '18.19', '19.20', '20.21', '21.22', '22.23', '23.24']
# terms = ['15.16', '16.17', '17.18', '18.19', '19.20', '21.22', '22.23','23.24', '24.25']

# for func in functions:
#     for term in terms:
#         print(f"Function: {func}, Term: {term}")
#         summary_df = get_summary(com, func, term)
#         display(summary_df)

In [29]:
import csv

functions = ['oGTa', 'iGTa', 'iGV', 'oGV', 'Total', 'oGTe', 'iGTe']
# terms = ['15.16', '16.17', '17.18', '18.19', '19.20','22.23', '23.24']
terms = ['15.16', '16.17', '17.18', '18.19', '19.20', '21.22', '22.23','23.24', '24.25']

# Open the CSV file in write mode
with open(f'{LEAD}Planning_Pivot.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    
    for func in functions:
        for term in terms:
            # Print and write the Function and Term identifiers
            print(f"Function: {func}, Term: {term}")
            
            # Get the summary DataFrame
            summary_df = get_summary(com, func, term)
            display(summary_df)
            
            # Write the Function and Term identifiers to the CSV
            csvwriter.writerow([f"Function: {func}, Term: {term}"])
            
            # Write the headers including the index name
            headers = ['Index'] + list(summary_df.columns)
            csvwriter.writerow(headers)
            
            # Write the DataFrame rows including the index
            for index, row in summary_df.iterrows():
                csvwriter.writerow([index] + list(row))
            
            # Add a blank line for separation (optional)
            csvwriter.writerow([])

print("Data written to summary_data.csv successfully.")


Function: oGTa, Term: 15.16


KeyError: "['EY Year'] not in index"